In [1]:
!pip install sacred
!pip install git+https://github.com/taylanot/SolidMechanicsTools.git
!git clone https://github.com/taylanot/DDSCT.git
!pip install SALib
!wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
!pip install gmsh
!pip install git+https://github.com/taylanot/gmshModel.git
!pip uninstall meshio -y
!pip install meshio==5.0.0
!pip install -U numpy
!pip install git+https://github.com/taylanot/gmshModel.git

     |████████████████████████████████| 106 kB 6.7 MB/s 
     |████████████████████████████████| 180 kB 43.3 MB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=0af15a2e275a8f753b66b89dca249061708b9b9f964fa4396c4769edae63fb16
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built py-cpuinfo
  Cloning https://github.com/taylanot/SolidMechanicsTools.git to /tmp/pip-req-build-xgf4u68c
  Running command git clone -q https://github.com/taylanot/SolidMechanicsTools.git /tmp/pip-req-build-xgf4u68c
  Created wheel for SolidMechanicsTools: filename=SolidMechanicsTools-0.0.1-py3-none-any.whl size=24926 sha256=64f0cdde62cca4c869da92f2361221aea554e26fb1298764ac0452415aa2bd78
  Stored in directory: /tmp/pip-ephem-wheel-cache-7gnm1mng/wheels/6e/12/aa/1719a34223ab59494941ac3

  Cloning https://github.com/taylanot/gmshModel.git to /tmp/pip-req-build-5xf8131p
  Running command git clone -q https://github.com/taylanot/gmshModel.git /tmp/pip-req-build-5xf8131p


In [2]:
from DDSCT.src.doe import *
from DDSCT.src.sim import *
from DDSCT.doe.SALib import SALibSampler
from DDSCT.main import *
import numpy as np
import torch
from dolfin import set_log_level
set_log_level(40)
from sacred import Experiment
from sacred.observers import FileStorageObserver


ex = Experiment('Bessa2017',interactive=True)

class Simpleton(torch.nn.Module):
    def __init__(self,set_up):
        super(Simpleton, self).__init__()

        for k, v in set_up.items():
            setattr(self, k, v)

        self.hidden = torch.nn.Linear(self.n_feature, self.n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(self.n_hidden, self.n_output)   # output layer
        self.activation = torch.nn.ReLU( )

    def forward(self, x):
        x = self.activation(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
@ex.config
def my_config():
    config = dict()
    config['doe'] = {'essentials':
            {
            'descriptors':{'F11':[-0.15, 1], 'F12':[-0.1,0.15],'F22':[-0.15, 1],\
                    'L':[1,2],'r':[0.1, 0.3],'Vf':[0.1, 0.2]},
            'application': 'SALib',
            'model': 'SALibSampler',
            'experiment_number': 10
            },
            'additional':
            {
            'method': 'sobol'
            }}


    config['simulation'] = {'essentials':
                {
                'application': 'SolidMechanicsTools',
                'model': 'Bessa2017',
                'output_keys': ['S11', 'S12', 'S22']
                }}

    config['machinelearning'] = {'essentials':
                {
                'application': 'PyTorch',
                'model': 'SimpleTraining',
                'output_name': 'MSE-Loss',
                'output_keys': ['train_loss','test_loss']
                },
                'additional':
                {
                'network': Simpleton,
                'network_parameters':{'n_feature':6, 'n_hidden':10, 'n_output':3},
                'split': 0.2,
                'optimizer': torch.optim.SGD,
                'optimizer_parameters':{'lr':0.01},
                'loss': torch.nn.MSELoss,
                'epoch': 100,
                }}
    NAME = "Bessa2017_DDSCT"
    ex.observers.append(FileStorageObserver(NAME))

@ex.capture
def get_info(_run):
    return _run._id, _run.experiment_info["name"]

@ex.main
def main(config,NAME):
    _id, _ = get_info()
    artifacts = os.path.join(NAME,'artifacts',_id)
    os.makedirs(artifacts)
    doe, _ = sample(config['doe'],path=os.path.join(artifacts))
    dataset, _ = simulate(doe, config['simulation'], num_workers=2, path=os.path.join(artifacts))
    surrogate = train(dataset, config['machinelearning'], path=os.path.join(artifacts))

ex.run()

INFO - Bessa2017 - Running command 'main'
INFO - Bessa2017 - Started run with ID "1"


-----------------------------------------------------
                       DOE INFO                      
-----------------------------------------------------


Sampling Application : SALib
Sampling Model       : SALibSampler
Number of Descriptors: 6
Number of Experiments: 10
DOE Dataset          : <xarray.Dataset>
Dimensions:            (descriptor: 6, doe: 10)
Coordinates:
  * doe                (doe) int64 0 1 2 3 4 5 6 7 8 9
  * descriptor         (descriptor) <U3 'F11' 'F12' 'F22' 'L' 'r' 'Vf'
Data variables:
    Running_Variables  (doe, descriptor) float64 -0.15 -0.1 ... 0.1125 0.1813




WARNING - gmshModel.Visualization.GeometryVisualization - The geometry visualization relies on the pythonocc package. To visualize the model geometry, install pythonocc.
INFO - numexpr.utils - NumExpr defaulting to 2 threads.


Creating the RVEs...-/|\-/|\-/
-----------------------------------------------------
                  SIMULATION PROGRESS                
-----------------------------------------------------


  0%|          | 0/10 [00:00<?, ?it/s]

...

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25 - FFC - Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO - FFC - Compiling form ffc_form_dc59d844c4fe050fb59f48ec3ae62171fe771f32

INFO - FFC - Compiler stage 1: Analyzing form(s)
INFO - FFC - -----------------------------------
DEBUG - FFC -   Preprocessing form using 'uflacs' representation family.
INFO - FFC -   
INFO - FFC -   Geometric dimension:       2
  Number of cell subdomains: 0
  Rank:                      2
  Arguments:                 '(v_0, v_1)'
  Number of coefficients:    0
  Coefficients:              '[]'
  Unique elements:           'Tensor<(2, 2) x DG0(?,?)>, Vector<2 x CG1(?,?)>'
  Unique sub elements:       'Tensor<(2, 2) x DG0(?,?)>, Vector<2 x CG1(?,?)>, DG0(?
                             ,?), CG1(?,?)'
  
INFO - FFC -   representation:    auto --> uflacs
INFO - FFC -   quadrature_rule:   auto --> default
INFO - FFC -   quadrature_degree: auto --> 0
INFO - FFC -   quadrature_degree: 0
INFO - FFC -   
INFO - FFC - Compiler stage 1

-----------------------------------------------------
                  SIMULATION INFO                    
-----------------------------------------------------


Simulation Application   : SolidMechanicsTools
Simulation Model         : Bessa2017
Simulation DOE           : <xarray.Dataset>
Dimensions:            (descriptor: 6, doe: 10, outputs: 3)
Coordinates:
  * doe                (doe) int64 0 1 2 3 4 5 6 7 8 9
  * descriptor         (descriptor) <U3 'F11' 'F12' 'F22' 'L' 'r' 'Vf'
  * outputs            (outputs) <U3 'S11' 'S12' 'S22'
Data variables:
    Running_Variables  (doe, descriptor) float64 -0.15 -0.1 ... 0.1125 0.1813
    RVE_Directory      (doe) object 'Bessa2017_DDSCT/artifacts/1/rves/Vf:10-2...
    Results            (doe, outputs) float64 -79.81 -48.15 ... 8.027 92.48
Simulation Dataset       : <xarray.Dataset>
Dimensions:            (descriptor: 6, doe: 10, outputs: 3)
Coordinates:
  * doe                (doe) int64 0 1 2 3 4 5 6 7 8 9
  * descriptor         (descrip

100%|██████████| 100/100 [00:00<00:00, 600.52it/s]
INFO - Bessa2017 - Completed after 0:00:36


-----------------------------------------------------
               MACHINE LEARNING INFO                 
-----------------------------------------------------


Machine Learning Application : PyTorch
Machine Learning Model       : SimpleTraining
Machine Learning Dataset     : <xarray.Dataset>
Dimensions:            (descriptor: 6, doe: 10, outputs: 3)
Coordinates:
  * doe                (doe) int64 0 1 2 3 4 5 6 7 8 9
  * descriptor         (descriptor) <U3 'F11' 'F12' 'F22' 'L' 'r' 'Vf'
  * outputs            (outputs) <U3 'S11' 'S12' 'S22'
Data variables:
    Running_Variables  (doe, descriptor) float64 -0.15 -0.1 ... 0.1125 0.1813
    RVE_Directory      (doe) object 'Bessa2017_DDSCT/artifacts/1/rves/Vf:10-2...
    Results            (doe, outputs) float64 -79.81 -48.15 ... 8.027 92.48
Machine Learning Results     : <xarray.Dataset>
Dimensions:    (MSE-Loss: 2, tag: 100)
Coordinates:
  * tag        (tag) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
  * MSE-Loss   (M